In [1]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid="65955050144%3ARAdn9XAbejZtkT%3A18%3AAYfaudMUGTwXC6MNiBn-EaFCV701ZmbgsRdCw6xuBA")

def get_follower_data(data:list):
    result = []
    for d in data:
        node = d.get("node", {})
        result.append({
            "pk": node.get("pk", ""),
            "username": node.get("username", ""),
            "follower_count": node.get("follower_count", "")
        })
    return result


def get_all_follower(userid:str):
    results = []
    response = t.retrieve_follower_by_userid(userid)
    data = response.get("data", {})
    data = data.get("user", {})
    data = data.get("followers",{})
    results.extend(get_follower_data(data.get("edges", [])))
    has_next_page = data.get("page_info", {}).get("has_next_page", False)
    after = data.get("page_info", {}).get("end_cursor")
    
    while has_next_page:
        response = t.retrieve_follower_by_userid_after(userid, after)
        data = response.get("data", {})
        data = data.get("fetch__XDTUserDict", {})
        data = data.get("followers",{})
        results.extend(get_follower_data(data.get("edges", []))) 
        has_next_page = data.get("page_info", {}).get("has_next_page", False)
        
    return results

In [18]:
import threading
from retrieve_data_by_query import RetrieveDataByQuery

t = RetrieveDataByQuery(sessionid="65955050144%3ACYLJ62xy3SlmZY%3A21%3AAYczdKeVZcK3hiLzXoCohMHegWecE3FIvmjZ_Myz8w")

def fetch_followers_thread(userid: str, i: int, results: list, lock: threading.Lock):
    after = str(25 * i)
    response = t.retrieve_follower_by_userid_after(userid, after)
    data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
    batch_results = get_follower_data(data.get("edges", []))
    
    with lock:
        results.extend(batch_results)

def get_all_follower(userid: str):
    results = []
    response = t.retrieve_follower_by_userid(userid)
    data = response.get("data", {}).get("user", {}).get("followers", {})
    results.extend(get_follower_data(data.get("edges", [])))
    
    has_next_page = data.get("page_info", {}).get("has_next_page", False)
    i = 1
    lock = threading.Lock()
    threads = []
    
    while has_next_page:
        print(i)
        thread = threading.Thread(target=fetch_followers_thread, args=(userid, i, results, lock))
        threads.append(thread)
        thread.start()
        i += 1
        
        response = t.retrieve_follower_by_userid_after(userid, str(25 * i))
        data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
        has_next_page = data.get("page_info", {}).get("has_next_page", False)
    
    for thread in threads:
        thread.join()
    
    return results


In [19]:
t.retrieve_follower_by_userid_after("187264532",'25')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
import json
import requests
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid="65955050144%3ACYLJ62xy3SlmZY%3A21%3AAYczdKeVZcK3hiLzXoCohMHegWecE3FIvmjZ_Myz8w")


data = {
            "_comet_req": "29",
            "lsd": t.api_token,
            "_spin_r": "101964087",
            "_spin_b": "trunk",
            "_spin_t": "1738088890",
            "fb_api_caller_class": "RelayModern",
            "fb_api_req_friendly_name": "BarcelonaSearchResultsRefetchableQuery",
            "variables": json.dumps(
                {
                    "first": 25,
                    "after": "50",
                    "id": "187264532",
                    "__relay_internal__pv__BarcelonaIsLoggedInrelayprovider": True,
                    "__relay_internal__pv__BarcelonaIsCrawlerrelayprovider": False,
                    "__relay_internal__pv__BarcelonaHasDisplayNamesrelayprovider": False,
                }
            ),
            "server_timestamps": "true",
            "doc_id": "9226067564176291",
        }
cookies = {
            "sessionid": "71441509544%3A9NroGpYJCtlrjn%3A1%3AAYe0D7Oa0fAH8aWyYbNn37a64dbS-vNqOjYqoxqPtA",
            # "sessionid": "65955050144%3ACYLJ62xy3SlmZY%3A21%3AAYczdKeVZcK3hiLzXoCohMHegWecE3FIvmjZ_Myz8w"
            # "sessionid": "65955050144%3AS1xuKkl6W1kKqB%3A23%3AAYcjBamMFOs7TY1uYVsrP-gAou7-TriJJsp8l1NgLA",
            # "sessionid": "71441509544%3AbPWmQfLc5vQhzF%3A7%3AAYfgsFzvtyS88WCjLlPNxUHfxAyfSs-PMk0cunbBMA"
            # "sessionid":"1653774166%3A9uDdOCkxtjfjYr%3A25%3AAYf2jGK4vIx5W_dkE5m8a7PpoN4LA2D5CxUpR4C7BA"
        }
response = requests.post(
            t.url, headers=t.default_headers, data=data, cookies=cookies
)
response.json()

{'data': {'fetch__XDTUserDict': {'followers': {'edges': [{'node': {'pk': '1235112373',
       'text_post_app_is_private': False,
       'friendship_status': {'followed_by': False,
        'following': False,
        'outgoing_request': False},
       'profile_pic_url': 'https://scontent-fra3-1.cdninstagram.com/v/t51.2885-19/464760996_1254146839119862_3605321457742435801_n.png?stp=dst-jpg_e0_s150x150_tt6&cb=8577c754-c2464923&_nc_ht=scontent-fra3-1.cdninstagram.com&_nc_cat=1&_nc_ohc=w2biA7W10AIQ7kNvgHqPkQJ&_nc_gid=404848de66eb45f1a3706d8816c00632&edm=AGqCYasBAAAA&ccb=7-5&ig_cache_key=YW5vbnltb3VzX3Byb2ZpbGVfcGlj.3-ccb7-5-cb8577c754-c2464923&oh=00_AYDROztBljuL_Tsq5vvzim8mzqiBTGhAdfIzejj87_dCDA&oe=67A06AA8&_nc_sid=6c5dea',
       'username': 'theworkroomict',
       'follower_count': 1823,
       'profile_context_facepile_users': [],
       'is_verified': False,
       'full_name': 'The Workroom',
       'has_onboarded_to_text_post_app': True,
       'id': '1235112373',
       '__typename'